In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# get dataframes
df_pid = pd.read_csv('./hospiedata/data/cleaned/id_to_attrs_no_multi_ehr.csv')
df_care = pd.read_csv('./hospiedata/data/care_translated_all_cols.csv', parse_dates=['TimeStamp_Insert'])
df_ht = pd.read_csv('./hospiedata/data/cleaned/therapy_cleaned.csv')

C:\Users\prate\Anaconda3\envs\det2\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,11,13,15,16,17,18,19,20,22,23,24,25,26,27,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\prate\Anaconda3\envs\det2\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# fix dtype errors
# df_pid.dtypes

In [4]:
# df_care.dtypes
df_care.columns

Index(['Unnamed: 0', 'IDEHR', 'Author_Insert', 'TimeStamp_Insert', 'IDAccess',
       'EHRType', 'PatientID', 'Note_Non_Rilevabile', 'IDGESTIONE_AUSILI',
       'ds_ncons', 'ds_nbolla', 'dt_consegna', 'ds_nritiro', 'dt_ritiro',
       'opt_annulla_consegna', 'ds_note_x', 'dt_Ric_consegna',
       'dt_ric_cons_forn', 'dt_ric_ritiro', 'dt_ric_ritiro_forn',
       'opt_ausilio', 'IDTERAPIE_OUTPAT_VIDAS', 'ds_altro_farmaco', 'ds_dose',
       'opt_via_di_somm', 'ds_ora', 'dt_data_inizio', 'ds_note_y',
       'opt_pregressa', 'opt_somm_terapia', 'opt_estemporanea', 'opt_termina',
       'opt_somm_in_pompa', 'opt_farmaco', 'Note_al_bisogno'],
      dtype='object')

In [5]:
# df_ht.dtypes

In [6]:
# split the dframe into considering only those IDEHR whose values appear in both 
print(len(df_pid.idehr.unique()))
print(len(df_care.IDEHR.unique()))
pid_set = set(df_pid.idehr.unique())
care_set = set(df_care.IDEHR.unique())
common_set = list(care_set.intersection(pid_set))
print(len(common_set))

6790
4300
4077


In [7]:
# get the new care dframe
print(df_care.shape)
df_care = df_care[df_care['IDEHR'].isin(common_set)]
print(df_care.shape)

(105519, 35)
(102404, 35)


In [8]:
df_care.head(2)

,Unnamed: 0,IDEHR,Author_Insert,TimeStamp_Insert,IDAccess,EHRType,PatientID,Note_Non_Rilevabile,IDGESTIONE_AUSILI,ds_ncons,...,ds_ora,dt_data_inizio,ds_note_y,opt_pregressa,opt_somm_terapia,opt_estemporanea,opt_termina,opt_somm_in_pompa,opt_farmaco,Note_al_bisogno
18,18,203.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:19:04,9860.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,20,209.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:37:20,9864.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_care.opt_farmaco.value_counts()

other (see notes) # 2004                                                                                      9018
furosemide (25 mg lasix tablets) # 1223                                                                       2286
dexamethasone (soldesam os gtt 0-2% gtt) # 1446                                                               1938
morphine hydrochloride (10 mg morphine hydrochloride fl) # 1598                                               1723
scopolamine butylbromide (buscopan 20mg / ml fl) # 997                                                        1549
                                                                                                              ... 
timolol (0.25% timoptol coll) # 1974                                                                             1
timolol (droptimol 0-25% coll) # 1972                                                                            1
venlafaxine (150 mg efexor cps rp) # 1918 = 0                                   

In [10]:
# create the new columns
df_care['DrugName'] = ''
df_care['Dose'] = np.nan
df_care['DrugRef'] = np.nan
df_care['DrugNote'] = ''

In [11]:
df_care.head(2)

,Unnamed: 0,IDEHR,Author_Insert,TimeStamp_Insert,IDAccess,EHRType,PatientID,Note_Non_Rilevabile,IDGESTIONE_AUSILI,ds_ncons,...,opt_somm_terapia,opt_estemporanea,opt_termina,opt_somm_in_pompa,opt_farmaco,Note_al_bisogno,DrugName,Dose,DrugRef,DrugNote
18,18,203.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:19:04,9860.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
20,20,209.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:37:20,9864.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,


In [12]:
# get the drug/diagnosis reference number
df_care['DrugRef'] = df_care.opt_farmaco.str.findall('(?<=#.).[0-9]+').to_frame()
df_care['DrugRef'] = df_care['DrugRef'].apply(lambda x: x if isinstance(x, float) else float(x[0]))
print(df_care['DrugRef'][105517])
# print(df_care.DrugRef.unique())

1626.0


In [13]:
# get the drug/active ingredient name
df_care['DrugName'] = df_care.opt_farmaco.str.findall('(.+).\(').to_frame()
df_care['DrugName'] = df_care['DrugName'].apply(lambda x: x if isinstance(x, float) else x[0])
print(df_care['DrugName'][105517])

oxycodone / acetaminophen


In [14]:
df_care['opt_farmaco'][105517]

'oxycodone / acetaminophen (depalgos 10 + 325 mg tablets rev) # 1626'

In [15]:
# capture everything within the brackets
df_care['DrugNote'] = df_care.opt_farmaco.str.findall('\((.+)\)').to_frame()
df_care['DrugNote'] = df_care['DrugNote'].apply(lambda x: x if isinstance(x, float) else x[0])
print(df_care['DrugNote'][105517])

depalgos 10 + 325 mg tablets rev


In [16]:
df_care.tail(10)

,Unnamed: 0,IDEHR,Author_Insert,TimeStamp_Insert,IDAccess,EHRType,PatientID,Note_Non_Rilevabile,IDGESTIONE_AUSILI,ds_ncons,...,opt_somm_terapia,opt_estemporanea,opt_termina,opt_somm_in_pompa,opt_farmaco,Note_al_bisogno,DrugName,Dose,DrugRef,DrugNote
105509,93218,11942.0,groff liliana - grflln55a70g452v,2017-07-31 13:09:10,82616.0,amb,43953.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,morphine sulfate (10 mg oramorph 5 ml flac os)...,"if pain and or wheezing, repeatable.",morphine sulfate,NaN,1604.0,10 mg oramorph 5 ml flac os
105510,93219,11942.0,groff liliana - grflln55a70g452v,2017-07-31 13:09:13,82616.0,amb,43953.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,fentanyl (durogesic tts 12 mcg / hour) # 1647,NaN,fentanyl,NaN,1647.0,durogesic tts 12 mcg / hour
105511,93220,11942.0,groff liliana - grflln55a70g452v,2017-07-31 13:09:16,82616.0,amb,43953.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,morphine hydrochloride (10 mg morphine hydroch...,if ..ripetibile pain and or shortness of breath.,morphine hydrochloride,NaN,1598.0,10 mg morphine hydrochloride fl
105512,93221,11942.0,groff liliana - grflln55a70g452v,2017-07-31 13:09:25,82616.0,amb,43953.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,+ acetaminophen codeine sulfate (30 mg + 500 t...,up to 4 sachets a day,+ acetaminophen codeine sulfate,NaN,1634.0,30 mg + 500 tachidol bust
105513,93222,11942.0,groff liliana - grflln55a70g452v,2017-07-31 13:09:31,82616.0,amb,43953.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,other (see notes) # 2004,NaN,other,NaN,2004.0,see notes
105514,93223,11906.0,prison silvia - prgslv79a63h264p,2017-07-31 13:12:53,82618.0,amb,43948.0,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,pregabalin (lyrica 25 mg cps) # 1771,NaN,pregabalin,NaN,1771.0,lyrica 25 mg cps
105515,93224,11906.0,prison silvia - prgslv79a63h264p,2017-07-31 13:12:57,82618.0,amb,43948.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,other (see notes) # 2004,NaN,other,NaN,2004.0,see notes
105516,93225,11906.0,prison silvia - prgslv79a63h264p,2017-07-31 13:13:00,82618.0,amb,43948.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,macrogol / sodium bic./sodio cl / kcl (13.8 g ...,NaN,macrogol / sodium bic./sodio cl / kcl,NaN,1035.0,13.8 g movicol bust
105517,93226,11906.0,prison silvia - prgslv79a63h264p,2017-07-31 13:13:03,82618.0,amb,43948.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,oxycodone / acetaminophen (depalgos 10 + 325 m...,NaN,oxycodone / acetaminophen,NaN,1626.0,depalgos 10 + 325 mg tablets rev
105518,93227,11906.0,prison silvia - prgslv79a63h264p,2017-07-31 13:13:06,82618.0,amb,43948.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,fluconazole (diflucan 200 mg cps) # 1531,NaN,fluconazole,NaN,1531.0,diflucan 200 mg cps


In [17]:
# create new dframe purely for the drugs used
df_drug = df_care[['DrugName', 'Dose', 'DrugRef', 'DrugNote']].drop_duplicates().sort_values(by=['DrugName'], ascending=True).reset_index()
# split the string by '+'
df_drug['t1:+'] = df_drug.DrugNote.str.findall('(.+)\+(.+)')
df_drug['t1:+'] = df_drug['t1:+'].apply(lambda x: x if isinstance(x, float) else x)
df_drug['Dose'] = df_drug.DrugNote.str.findall('.+\s(.+mg)')
df_drug['Dose'] = df_drug['Dose'].apply(lambda x: np.nan if isinstance(x, float) or not x else x[0])
# split the string by '/'
df_drug['t2:/'] = df_drug.DrugName.str.findall('(.+)\s/\s(.+)')
df_drug['t2:/'] = df_drug['t2:/'].apply(lambda x: x if isinstance(x, float) else x)
# split the doses for those entries containing two drugs in the same row
df_drug['t3:+d'] = df_drug.DrugNote.str.findall('(\d+)')
df_drug['t3:+d'] = df_drug['t3:+d'].apply(lambda x: x if isinstance(x, float) else x)
print(df_drug.shape)
df_drug.head(50)

(1042, 7)


,index,DrugName,Dose,DrugRef,DrugNote,t1:+,t2:/
0,24374,+ acetaminophen codeine sulfate,NaN,1634.0,30 mg + 500 tachidol bust,"[(30 mg , 500 tachidol bust)]",[]
1,32948,+ acetaminophen codeine sulfate,NaN,1633.0,30 mg + 500 coefferalgan bust,"[(30 mg , 500 coefferalgan bust)]",[]
2,53281,+ acetaminophen codeine sulfate,NaN,1632.0,30 mg tablets codamol 500+ rev,"[(30 mg tablets codamol 500, rev)]",[]
3,30847,acarbose,50 mg,1096.0,glucobay 50 mg tablets,[],[]
4,75998,acarbose,50 mg,1093.0,acarphage 50 mg tablets,[],[]
5,26800,acarbose,100 mg,1094.0,glicobase 100 mg tablets,[],[]
6,24965,acarbose,100 mg,1095.0,glucobay 100 mg tablets,[],[]
7,24966,acarbose,100 mg,1092.0,acarphage 100 mg tablets,[],[]
8,24352,acenocoumarol,1 mg,1126.0,sintrom tablets 1 mg,[],[]
9,35413,acenocoumarol,4 mg,1127.0,sintrom 4 mg tablets div,[],[]


In [18]:
# examples of where problems might occur during regex
print(df_care['opt_farmaco'][23959])
print(df_care['opt_farmaco'][23973])
print(df_care['opt_farmaco'][23679])

acetazolamide (diamox * 12 tablets 250 mg - 12 tablets 250 mg diamox *) # 2130 = 0
albuterol (ventolin * susp inal 200 erog 100 mcg - ventolin * susp inal 200 erog 100 mcg) # 2119 = 0
allopurinol (teva allopurinol italy - 300 mg tablets 30 tablets in blisters pvc / pvdc / al) # 1708 = 0


In [19]:
# crate a new dframe where the drugs having different trade names but the same active ingridient and dose are combined
df_drug_long = df_care[['IDEHR', 'TimeStamp_Insert', 'EHRType', 'ds_dose', 'opt_farmaco', 'DrugName', 'Dose', 'DrugRef', 'DrugNote']].reset_index().rename(columns={"index": "df_care_index"})
df_drug_long.head(5)

,df_care_index,IDEHR,TimeStamp_Insert,EHRType,ds_dose,opt_farmaco,DrugName,Dose,DrugRef,DrugNote
0,18,203.0,2015-06-15 13:19:04,amb,NaN,NaN,NaN,NaN,NaN,NaN
1,20,209.0,2015-06-15 13:37:20,amb,NaN,NaN,NaN,NaN,NaN,NaN
2,25,209.0,2015-06-22 10:35:58,amb,NaN,NaN,NaN,NaN,NaN,NaN
3,26,203.0,2015-06-22 10:38:14,amb,NaN,NaN,NaN,NaN,NaN,NaN
4,29,2536.0,2015-06-25 12:20:07,amb,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
# saved Sep 16, 2020